# IT Service Ticket Classification

## Objetivo

Desenvolver um sistema de classificação automática de tickets de suporte de TI que:
- **Entrada:** texto do ticket (string)
- **Saída:** `{"classe": "...", "justificativa": "..."}`

## Abordagem: RAG + LLM

Utilizamos **RAG (Retrieval Augmented Generation)** combinado com um **LLM (Large Language Model)**:

1. **Retrieval:** Buscar tickets similares no dataset de treino usando embeddings semânticos
2. **Augmented:** Enriquecer o prompt com exemplos relevantes
3. **Generation:** LLM classifica e justifica baseado no contexto

Esta abordagem foi escolhida porque:
- Temos um dataset grande (~48k tickets) com rótulos confiáveis
- Tickets similares tendem a pertencer à mesma classe
- Exemplos concretos ajudam a LLM a gerar justificativas precisas

## Notebooks

- `analise.ipynb`: Análise exploratória dos dados
- `main.ipynb` (este): Implementação e demonstração da solução

## 1. Carregamento e Preparação dos Dados

In [1]:
from classifier.data import load_dataset, train_test_split_stratified

In [2]:
# Carregar dataset
df, classes = load_dataset()
print(f"Total de tickets: {len(df):,}")
df.head()

Total de tickets: 47,837


,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous


In [3]:
# Classes obtidas do dataset
print(f"Classes ({len(classes)}):")
for c in classes:
    print(f"  - {c}")

Classes (8):
  - Access
  - Administrative rights
  - HR Support
  - Hardware
  - Internal Project
  - Miscellaneous
  - Purchase
  - Storage


In [4]:
# Split estratificado: treino para RAG, teste para avaliação (200 tickets)
train_df, test_df = train_test_split_stratified(df, test_size=200)

print(f"Treino: {len(train_df):,} tickets")
print(f"Teste:  {len(test_df)} tickets")

Treino: 47,637 tickets
Teste:  200 tickets


## 2. Por que RAG?

### Alternativas Consideradas

1. **Fine-tuning de LLM:** Treinar um modelo específico para a tarefa. Requer recursos computacionais significativos e risco de overfitting.

2. **Classificador tradicional (ML):** Usar TF-IDF + SVM ou similar. Boa precisão, mas não gera justificativas naturais.

3. **Zero-shot com LLM:** Usar apenas as descrições das classes. Funciona, mas perde a riqueza dos exemplos reais.

4. **RAG (escolhido):** Combina o melhor dos mundos - usa exemplos reais para guiar a classificação e aproveita a capacidade do LLM para justificar.

### Por que RAG é adequado para este problema?

Com base na análise exploratória (`analise.ipynb`):

- **Dataset grande e rotulado:** ~48k tickets com labels confiáveis permitem encontrar exemplos similares
- **Desbalanceamento:** Classes minoritárias (ex: Administrative rights ~3.7%) se beneficiam de exemplos específicos
- **Textos curtos:** Mediana de ~26 palavras facilita o processamento de múltiplos exemplos no prompt
- **Padrões de vocabulário:** Cada classe tem palavras-chave distintas que embeddings capturam bem

### Vantagens desta abordagem

1. **Justificativas de qualidade:** LLM vê exemplos reais e pode referenciar padrões similares
2. **Adaptável:** Novos tickets de treino melhoram automaticamente o retrieval
3. **Explicável:** Podemos inspecionar quais exemplos influenciaram a decisão

## 3. RAG - Retrieval de Tickets Similares

### Como funciona

1. **Embeddings:** Cada ticket é convertido em um vetor de 384 dimensões usando o modelo `all-MiniLM-L6-v2` (sentence-transformers). Este modelo foi treinado para capturar similaridade semântica.

2. **Similaridade de cosseno:** Para encontrar tickets similares, calculamos o cosseno do ângulo entre vetores. Valores próximos de 1 indicam alta similaridade.

3. **Tickets representativos:** Para cada classe, calculamos o "centróide" (média dos embeddings) e selecionamos o ticket mais próximo. Isso nos dá um exemplo típico de cada classe.

### Componentes

- `TicketRetriever.index()`: Gera embeddings para todos os tickets de treino
- `TicketRetriever.retrieve()`: Busca os K tickets mais similares
- `TicketRetriever.compute_representatives()`: Calcula tickets representativos por classe

In [5]:
from classifier.rag import TicketRetriever

In [6]:
# Indexar tickets de treino
retriever = TicketRetriever()
retriever.index(train_df)

# Calcular tickets representativos de cada classe (centróides)
representatives = retriever.compute_representatives()
print(f"\nTickets representativos calculados para {len(representatives)} classes")

Batches:   0%|          | 0/1489 [00:00<?, ?it/s]


Tickets representativos calculados para 8 classes


In [7]:
# Testar retrieval com um ticket do conjunto de teste
test_ticket = test_df.iloc[1]
query = test_ticket["Document"]
true_class = test_ticket["Topic_group"]

similar = retriever.retrieve(query, k=5)

print(f"Ticket de teste (classe real: {true_class}):")
print(f"{query}\n")
print("Tickets similares recuperados:")
for i, ticket in enumerate(similar, 1):
    print(f"\n{i}. [{ticket['class']}] (score: {ticket['score']:.3f})")
    print(f"   {ticket['text']}")

Ticket de teste (classe real: HR Support):
leaver time cards please action hi have advised couple times left can you please let know why still receiving these every week thanks sent october please action dear part established processes each required submit his weekly basis each friday for past week must be submitted for approval you receiving because you haven followed process please action possible below breakdown your please log accordingly training materials available here id if by each wednesday there still past period then your manager also be notified country manager for for date total days

Tickets similares recuperados:

1. [Miscellaneous] (score: 0.829)
   not submitted by leaver sent wednesday please action hi left st december can you please make also marked leaver app thanks sent monday please action dear part established processes each required submit his weekly basis each friday for past week must be submitted for approval you receiving because you haven followed process p

In [8]:
# Visualizar tickets representativos
print("Tickets representativos (mais próximos do centróide de cada classe):\n")
for class_name in sorted(representatives.keys()):
    t = representatives[class_name]
    print(f"[{class_name}] (score: {t['score']:.3f})")
    print(f"   {t['text']}\n")

Tickets representativos (mais próximos do centróide de cada classe):

[Access] (score: 0.773)
   password reset request maxim pm re confluence hi apologies please resetting client please regards confluence regards friday pm re confluence hello happens actions regards senior actuarial consultant commercial pricing ext st

[Administrative rights] (score: 0.813)
   windows upgrade upgrade analyst ext sent thursday october re upgrade good started process upgrade os faced with problem software center operating contains any updates can you help resolve problem sent tuesday october upgrade hello if you already upgraded your os please ignore also if you member teams please apply upgrade yet thank you you receiving because we approaching scheduled for forced upgrade assets migrated yet before performing upgrade your please document found here carefully attention screen below please contact or should you require any further assistance best regards ext

[HR Support] (score: 0.798)
   access thurs

## 4. Design dos Prompts

O prompt é a interface entre nosso sistema e a LLM. Um bom design de prompt é crucial para obter classificações precisas e justificativas úteis.

### Estrutura do Prompt

O prompt é dividido em duas partes:

1. **System prompt:** Define o papel da LLM (classificador), lista as classes válidas e especifica o formato de saída (JSON)

2. **User prompt:** Contém o ticket a classificar e os exemplos de contexto

### Parâmetros Configuráveis

| Parâmetro | Descrição | Valor Padrão |
|-----------|-----------|--------------|
| `K_SIMILAR` | Número de tickets similares do RAG | 5 |
| `reference_tickets` | Tickets representativos por classe | 1 por classe |

### Trade-offs

- **Mais exemplos similares:** Melhor contexto para classificação, mas aumenta tokens e custo
- **Tickets de referência:** Garante diversidade de classes, essencial para justificativas comparativas
- **Muitos exemplos:** Pode "poluir" o contexto e confundir a LLM

Vamos visualizar como os prompts são gerados:

In [9]:
from classifier.prompts import build_system_prompt, build_user_prompt

In [10]:
# Exemplo 1: System Prompt
system_prompt = build_system_prompt(classes)
print("=== SYSTEM PROMPT ===\n")
print(system_prompt)

=== SYSTEM PROMPT ===

Você é um classificador de tickets de suporte de TI.

Classifique o ticket em UMA das seguintes categorias:
- Access
- Administrative rights
- HR Support
- Hardware
- Internal Project
- Miscellaneous
- Purchase
- Storage

Responda APENAS com JSON no formato:
{"classe": "<categoria>", "justificativa": "<explicação curta de 1-2 frases>"}

A justificativa deve mencionar palavras-chave ou padrões do ticket que justificam a classificação.


In [11]:
# Exemplo 2: User Prompt COM tickets de referência
user_prompt_with_refs = build_user_prompt(query, similar, representatives)
print("=== USER PROMPT (com tickets de referência) ===\n")
print(user_prompt_with_refs)

=== USER PROMPT (com tickets de referência) ===

Classifique o seguinte ticket:

leaver time cards please action hi have advised couple times left can you please let know why still receiving these every week thanks sent october please action dear part established processes each required submit his weekly basis each friday for past week must be submitted for approval you receiving because you haven followed process please action possible below breakdown your please log accordingly training materials available here id if by each wednesday there still past period then your manager also be notified country manager for for date total days

## Tickets Similares
1. [Miscellaneous] not submitted by leaver sent wednesday please action hi left st december can you please make also marked leaver app thanks sent monday please action dear part established processes each required submi...
2. [Miscellaneous] missing timecards please action hi please you assist with below advised still issue thanks sen

In [12]:
# Exemplo 3: User Prompt SEM tickets de referência
# Útil quando queremos usar menos tokens ou quando os similares já são suficientes
user_prompt_no_refs = build_user_prompt(query, similar, reference_tickets=None)
print("=== USER PROMPT (sem tickets de referência) ===\n")
print(user_prompt_no_refs)

=== USER PROMPT (sem tickets de referência) ===

Classifique o seguinte ticket:

leaver time cards please action hi have advised couple times left can you please let know why still receiving these every week thanks sent october please action dear part established processes each required submit his weekly basis each friday for past week must be submitted for approval you receiving because you haven followed process please action possible below breakdown your please log accordingly training materials available here id if by each wednesday there still past period then your manager also be notified country manager for for date total days

## Tickets Similares
1. [Miscellaneous] not submitted by leaver sent wednesday please action hi left st december can you please make also marked leaver app thanks sent monday please action dear part established processes each required submi...
2. [Miscellaneous] missing timecards please action hi please you assist with below advised still issue thanks sen

## 5. Classificação com LLM

Com o prompt construído, enviamos para a LLM via OpenRouter. O modelo utilizado é o `gemini-2.0-flash-exp` (gratuito), que responde em formato JSON estruturado.

**Requer:** variável de ambiente `OPENROUTER_API_KEY`

In [13]:
from classifier.llm import TicketClassifier

In [14]:
# Inicializar classificador
classifier = TicketClassifier()

In [15]:
# Classificar o ticket de teste
result = classifier.classify(query, similar, classes, reference_tickets=representatives)

print(f"Classe real: {true_class}")
print(f"Classe predita: {result.classe}")
print(f"Justificativa: {result.justificativa}")
print(f"\nCorreto: {result.classe == true_class}")

Classe real: HR Support
Classe predita: HR Support
Justificativa: O ticket trata de um leaver (ex-funcionário) que não segue o processo de envio de cartões de tempo (time cards), e menciona comunicação repetida sobre o assunto, padrão típico de processos de RH.

Correto: True
